In [ ]:
!pip install transformers
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from numba import cuda
device = cuda.get_current_device() 
device.reset()


In [ ]:

import tensorflow as tf
import os
print(f"Tensorflow version: {tf.__version__}")

# Restrict TensorFlow to only allocate 4GBs of memory on the first GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    #tf.config.experimental.set_memory_growth(gpus[0], True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(f"The system contains '{len(gpus)}' Physical GPUs and '{len(logical_gpus)}' Logical GPUs")
  except RuntimeError as e:
    print(e)
else:
    print(f"Your system does not contain a GPU that could be used by Tensorflow!")

Tensorflow version: 2.12.0
The system contains '1' Physical GPUs and '1' Logical GPUs


In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import numpy as np
from sklearn.model_selection import train_test_split

from transformers import (TFBertForSequenceClassification, 
                          BertTokenizer)

from tqdm import tqdm

In [ ]:
data = pd.read_csv('dataset_sentences.csv')

# Transform positive/negative values to 1/0s
label_encoder = preprocessing.LabelEncoder()
data['sentiment'] = label_encoder.fit_transform(data['sentiment'])

X = (np.array(data['review']))
y = (np.array(data['sentiment']))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1, stratify=data["sentiment"])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.111, random_state=1)

print("Train dataset shape: {0}, \nTest dataset shape: {1} \nValidation dataset shape: {2}".format(X_train.shape, X_test.shape, X_val.shape))

Train dataset shape: (271,), 
Test dataset shape: (34,) 
Validation dataset shape: (34,)


In [ ]:
X_train, y_train = expand(X_train, y_train)
X_test, y_test = expand(X_test, y_test)
X_val, y_val = expand(X_val, y_val)

In [ ]:
def expand(x, y):
  final_x = []
  final_y = []

  for i, sample in enumerate(x):
    samples = sample.split("$")
    final_x += samples
    
    label = y[i]
    final_y += ([label] * len(samples))

  return np.array(final_x), np.array(final_y)

In [ ]:
bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-cased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pad_token=0
pad_token_segment_id=0
max_length=128

def convert_to_input(reviews):
  input_ids,attention_masks,token_type_ids=[],[],[]
  
  for x in tqdm(reviews,position=0, leave=True):
    inputs = bert_tokenizer.encode_plus(x,add_special_tokens=True, max_length=max_length)
    
    i, t = inputs["input_ids"], inputs["token_type_ids"]
    m = [1] * len(i)

    padding_length = max_length - len(i)

    i = i + ([pad_token] * padding_length)
    m = m + ([0] * padding_length)
    t = t + ([pad_token_segment_id] * padding_length)
    
    input_ids.append(i)
    attention_masks.append(m)
    token_type_ids.append(t)
  
  return [np.asarray(input_ids), 
            np.asarray(attention_masks), 
            np.asarray(token_type_ids)]

In [ ]:
X_test_input=convert_to_input(X_test)
X_train_input=convert_to_input(X_train)
X_val_input=convert_to_input(X_val)

100%|██████████| 152/152 [00:00<00:00, 1839.80it/s]


In [ ]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
  return {"input_ids": input_ids,
          "attention_mask": attention_masks,
          "token_type_ids": token_type_ids},y

train_ds = tf.data.Dataset.from_tensor_slices((X_train_input[0],X_train_input[1],X_train_input[2],y_train)).map(example_to_features).shuffle(100).batch(12).repeat(5)
val_ds=tf.data.Dataset.from_tensor_slices((X_val_input[0],X_val_input[1],X_val_input[2],y_val)).map(example_to_features).batch(12)
test_ds=tf.data.Dataset.from_tensor_slices((X_test_input[0],X_test_input[1],X_test_input[2],y_test)).map(example_to_features).batch(12)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
print("Fine-tuning BERT")
bert_history = bert_model.fit(train_ds, epochs=3, validation_data=val_ds)

Fine-tuning BERT
Epoch 1/3
475/475 [==============================] - 233s 370ms/step - loss: 0.6799 - accuracy: 0.5914 - val_loss: 0.6351 - val_accuracy: 0.8026
Epoch 2/3
475/475 [==============================] - 161s 339ms/step - loss: 0.6756 - accuracy: 0.5850 - val_loss: 0.6329 - val_accuracy: 0.8026
Epoch 3/3
475/475 [==============================] - 160s 336ms/step - loss: 0.6677 - accuracy: 0.5933 - val_loss: 0.6942 - val_accuracy: 0.1974


In [ ]:
results_true = test_ds.unbatch()
results_true = np.asarray([element[1].numpy() for element in results_true])
print(results_true)

[0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
results = bert_model.predict(test_ds)
print(f"Model predictions:\n {results.logits}")

results_predicted = np.argmax(results.logits, axis=1)

9/9 [==============================] - 4s 110ms/step
Model predictions:
 [[-0.15677695 -0.16025719]
 [-0.15677437 -0.16025986]
 [-0.1567756  -0.16025771]
 [-0.15677924 -0.16025887]
 [-0.1567742  -0.16026358]
 [-0.15677762 -0.1602569 ]
 [-0.15678588 -0.16025688]
 [-0.15678132 -0.16024905]
 [-0.1567759  -0.16025743]
 [-0.15676999 -0.16026603]
 [-0.15677546 -0.1602696 ]
 [-0.15678693 -0.16025935]
 [-0.15677857 -0.1602553 ]
 [-0.15677734 -0.16025612]
 [-0.15677461 -0.16025992]
 [-0.15677409 -0.16026172]
 [-0.1567807  -0.16025344]
 [-0.15676998 -0.16026618]
 [-0.15676713 -0.16027115]
 [-0.15676902 -0.16027142]
 [-0.15677321 -0.16026172]
 [-0.15677994 -0.16025168]
 [-0.15676959 -0.16026744]
 [-0.15676375 -0.16027533]
 [-0.15677366 -0.1602602 ]
 [-0.156772   -0.16026257]
 [-0.1567676  -0.16027202]
 [-0.15677865 -0.1602587 ]
 [-0.15677324 -0.16026117]
 [-0.15676908 -0.16026907]
 [-0.1567746  -0.16026124]
 [-0.15677509 -0.16026008]
 [-0.15676947 -0.16026814]
 [-0.15677981 -0.1602526 ]
 [-0.1567

In [ ]:
results_predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print(f"F1 score: {f1_score(results_true, results_predicted)}")
print(f"Accuracy score: {accuracy_score(results_true, results_predicted)}")

F1 score: 0.0
Accuracy score: 0.32710280373831774


In [ ]:
predicted_baseline = np.ones(len(results_true))
print(f"F1 score: {f1_score(results_true, predicted_baseline)}")
print(f"Accuracy score: {accuracy_score(results_true, predicted_baseline)}")

F1 score: 0.8044692737430167
Accuracy score: 0.6728971962616822


In [ ]:
# SAVING YOUR MODEL
bert_model.save_pretrained('/content')

# LOADING YOUR MODEL
bert_model = TFBertForSequenceClassification.from_pretrained('./')

Some layers from the model checkpoint at ./ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
